In [ ]:
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import torch
import torch_geometric
import torch
import os
import os.path as osp
import numpy as np
from tqdm import tqdm as tqdm
#from tqdm.notebook import tqdm

print("pyg:",torch_geometric.__version__)
print("torch:",torch.__version__)

In [ ]:
import glob
import matplotlib.pyplot as plt

In [ ]:
eb_scale=50.
m0_scale=18

In [ ]:
def transform_y(y):
    return (y)/m0_scale

def inv_transform(y):
    return (y*m0_scale)

In [ ]:
data_list_pi=[]
raw_dir= '/export/home/phys/aharilal/MassRegressor/a2gg-regression/GNNscripts/GNNDATA/Pi0_graphs/Pi0_UL17/EBTZ_rwcl_xyz/*/'
fnamelistt = [filepath for filepath in glob.glob(raw_dir+'data*.pt')]
for i in tqdm(fnamelistt):
    idx = torch.load(i)
    idx.genmass = transform_y(idx.pmass)
    #print(idx.genmass)
    #idx.X= idx.x
    #print(idx.x.shape)
    idx.x[:,0]=idx.x[:,0]/eb_scale
    idx.x[:,1]=idx.x[:,1]/eb_scale
    #idx.x[:,2]=idx.x[:,2] + idx.pscieta
    #idx.x[:,3]=idx.x[:,3] + idx.psciphi
    data_list_pi.append(idx)

In [ ]:
### pmass, pscieta, psciphi : tensor names
# import pho data
hi,lo = -2., 0.


import glob
raw_dir= '/export/home/phys/aharilal/MassRegressor/a2gg-regression/GNNscripts/GNNDATA/Gamma_graphs/Gamma_UL17/EBTZ_rwcl_xyz/0000/'#'/data_CMS/cms/sghosh/ECALGNNDATA/GRAPHS_ietaiphi/GenGamma/'
fnamelist = [filepath for filepath in glob.glob(raw_dir+'data*.pt')]
data_list_pho = []
ctr = 0
for i in tqdm(fnamelist):
    idx = torch.load(i)
    idx.genmass =  torch.tensor((hi-lo)*np.random.random_sample(size=(1,1)) + lo , dtype=torch.float32)
    #print(idx.genmass)
    #idx.x[] = idx.x
    idx.x[:,0] = idx.x[:,0]/eb_scale
    idx.x[:,1] = idx.x[:,1]/eb_scale
    #idx.x[:,2]=idx.x[:,2] + idx.pscieta
    #idx.x[:,3]=idx.x[:,3] + idx.psciphi
    idx.genmass = transform_y(idx.genmass)
    #print(idx.genmass)
    
    data_list_pho.append(idx)
    #data_list.append(torch.load(i))
    
    if ctr > 71000 :
        break
    ctr +=1
        
        
print(data_list_pho[0].pmass,data_list_pho[0].genmass)
totalevpho = len(data_list_pho)
#trainev ='''

In [ ]:
plot_dir='/export/home/phys/aharilal/MassRegressor/a2gg-regression/PLOTS/GNN_plots/UL17_GNN_scale50_WEBTZ_rwcl_xyz'
#for d in ['MODELS','PLOTS']:
if not os.path.isdir(plot_dir):
    os.makedirs(plot_dir)

checkpoint_dir = '/export/home/phys/aharilal/MassRegressor/a2gg-regression/MODELS/UL17_GNN_scale50_WEBTZ_rwcl_xyz'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
#del data_list_comb
trainfrac = 0.8

totalevpi = len(data_list_pi)
trainpi =  int(0.8*totalevpi) #151000

trainpho = int(0.8*len(data_list_pho)) #27768  #int((trainpi/18.0)*(lo-hi))*2

print("totalevpi, trainpi, trainpho :",totalevpi,trainpi,trainpho)

import random

data_list_photest = data_list_pho  #[]
random.shuffle(data_list_pi)
data_list_train = data_list_pi[:trainpi] + data_list_photest[:trainpho]
random.shuffle(data_list_train)
totaltrain = len(data_list_train)
print("total train evs:",totaltrain)


valpi = len(data_list_pi[trainpi:]) #32895
valpho = len(data_list_pho[trainpho:]) #3086#3000##int((valpi/18.0)*(lo-hi))

data_list_val = data_list_pi[trainpi:] #+ data_list_photest[trainpho:trainpho+valpho]

print("valpi,valpho,len(data_list_val) :",valpi,valpho,len(data_list_val))


import torch_geometric
ntrainbatch = 200
ntestbatch = 200
trainloader = torch_geometric.data.DataLoader(data_list_train, batch_size=ntrainbatch)
testloader = torch_geometric.data.DataLoader(data_list_val, batch_size=ntestbatch)
#batch_size = ntrainbatch
epoch_size = len(data_list_train)
print("epoch size,batch_size:",epoch_size,ntrainbatch)

In [ ]:
import os.path as osp

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
#from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm as tqdm

from models.GNN_DRN import DynamicReductionNetwork
from utils.load_scheduler import CyclicLRWithRestarts

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drn = DynamicReductionNetwork(input_dim=7,hidden_dim=40,k=16,output_dim=1,norm=torch.tensor([1.,1., 1./64., 1./64.,1./64., 1./64., 1./64.]))
        
    def forward(self, data):
        logits = self.drn(data)
        #return F.softplus(logits)
        return logits
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)
scheduler = CyclicLRWithRestarts(optimizer, ntrainbatch, epoch_size, restart_period=80, t_mult=1.2, policy="cosine")
#criterion = torch.nn.MSELoss()

#lossmse = nn.MSELoss()
def resoloss2(output,truth):
    batch_size = output.size()[0]
    #mse = F.mse_loss(output, truth, reduction='mean')
    #mse = torch.sum((output-truth)**2/truth)/batch_size
    #mse = torch.sum(((output-truth)/truth)**2)/batch_size
    mse = torch.mean(torch.abs((output - truth)))
    
    #res = 
    return (mse)


#model.train()



In [ ]:
def train(epoch):
    model.train()
    torch.cuda.empty_cache()
    scheduler.step()
    loss = []
    for data in tqdm(trainloader):
            #print("\n\nScaled data: ")
            #print(data.X[:,0].flatten())
            #print("\n Unscaled data: ")
            #print(data.x[:,0].flatten())
            data = data.to(device)
            #print("\n train data= ", data)
            optimizer.zero_grad()
            result = inv_transform(model(data))
            truthv = inv_transform(data.genmass.flatten())
            #print("train result: ",result)
            #print("original mass: ",truthv)
            lossc = resoloss2(result, truthv)
            #print("result:",result)
            #print("truth:",data.z)
            #lossc = categorical_loss(result, data.z, 1.0)
#            lossc = categorical_loss_only(result, data.z)
#            mse = F.mse_loss(result, data.y, reduction='mean')
#            mse = criterion(result, data.y)
#            print('result, y:',result,data.y)
#            print('crit, orig:',criterion(result, data.y),F.mse_loss(result, data.y, reduction='mean'))
            loss.append(lossc.item()) 
            lossc.backward()
#            print(mse)
            optimizer.step()
            scheduler.batch_step()
    print( 'batches for train:',len(loss)) 
#    print('loss',loss)
    print('train loss:',np.mean(np.array(loss)))
    return np.mean(np.array(loss))
    #del loss #memtest
#    print(data)


In [ ]:
from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as scs
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
#%matplotlib inline

def gaussian(x,  mean,a, sigma):
    return a * np.exp(-((x - mean)**2 / (2 * sigma**2)))

def evaluate(epoch):
        """"Evaluate the model"""
        model.zero_grad()
        torch.cuda.empty_cache()
        model.eval()
#        loss = []
#        frac = []

        pred = []
        true = []
        loss= []
        
        correct = 0
        predc = []
        truec = []
        for data in tqdm(testloader):
            data = data.to(device)        
            result = inv_transform(model(data))
            truthv = inv_transform(data.genmass.flatten())
            #print("eval result: ",result)
            #print("original mass: ",truthv)
            lossc = resoloss2(result, truthv)
#            print (result.item(),data.y.item())
#            frac.append((result.item() - data.y.item())/data.y.item())
            loss.append(lossc.item())

            for i in result:
                pred.append(i.detach().cpu())
                #predc.append(i.detach().cpu().argmax())
            for i in truthv.detach():
                true.append(i.detach().cpu())
                #truec.append(i.detach().cpu())
            
            
        #print(predc,truec)
        #predc = np.array(predc)
        #truec = np.array(truec)
        #print("accuracy  :",np.equal(predc,truec).sum()/len(truec))
        
        print('batches for test:', len(loss)) 
        print('test loss:',np.mean(np.array(loss)))
#        fracarr = np.array(frac

        
        preda = np.array(pred)
        truea = np.array(true)
        #preda = preda[:,2] ### added
        #truea = truea[:,1] ### added
        fracarr = (preda - truea)/truea
        #print(preda,truea,fracarr)
        print('pred - true / true mean:',(np.mean(fracarr)))
        print('pred - true / true std:',(np.std(fracarr)))
        (mu, sigma) = norm.fit(fracarr)
        print('mu,sig:',mu,sigma)
        

        #bin_heights, bin_borders, _ = plt.hist(fracarr,range=[-2,2], bins=100, label='histogram')
        #bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
        
        bins =  np.linspace(0,18,19)

        fig, axs = plt.subplots(6,3, figsize=(40, 40), facecolor='w', edgecolor='k')
        axs = axs.ravel()

        for i in tqdm(range (bins.size - 1)):
            predaa = preda[(truea >bins[i]) & (truea <bins[i+1]) ]
            trueaa = truea[(truea >bins[i]) & (truea <bins[i+1]) ]
            fracarr = (predaa - trueaa)/trueaa
            #if (fracarr < 0):
            axs[i].hist(fracarr,bins=100,range=[-4,4],)
            axs[i].set_xlabel('pred - true / true')
            axs[i].set_ylabel('counts')
            axs[i].set_title(str(bins[i])+" to "+str(bins[i+1]))
            #print (vals)
        if (epoch%5==0):
            plt.savefig('%s/mt-mp_frac_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()
        

        from matplotlib.pyplot import figure
        figure(figsize=(30, 30), dpi=40)
        #plt.rcParams['axes.labelsize'] = 36
        #plt.rcParams['axes.titlesize'] = 36
        #plt.hist2d(truea,preda,bins=200)
        plt.scatter(truea,preda,alpha=0.4)
        plt.plot([-4,18], [-4,18], 'k-')
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.xlim([-2, 18])
        plt.ylim([-2, 20])
        plt.axvline(x=1.0,c='r')
        plt.axhline(y=1.0,c='r')
        #plt.show()
        if (epoch%5 ==0):
            plt.savefig('%s/mpredVsmtrue_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()
        return np.mean(np.array(loss))
        #del pred,true,loss,preda,truea,fracarr  #memtest

In [ ]:

nepoch=500
os.makedirs(checkpoint_dir, exist_ok=True)
best_loss = 99999999
losst = []
lossv = []
epochs = []
for epoch in range(nepoch):
    print ('epoch:',epoch)
    losst.append(train(epoch))
    loss_epoch = evaluate(epoch)
    lossv.append(loss_epoch)
    epochs.append(epoch)
    checkpoint = {
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }
    
    checkpoint_file = 'model_epoch_%i.pth.tar' % ( epoch )
    torch.save(checkpoint,
                   os.path.join(checkpoint_dir,checkpoint_file ))
    if loss_epoch < best_loss:
        best_loss = loss_epoch
        print('new best test loss:',best_loss)
        torch.save(checkpoint,
                   os.path.join(checkpoint_dir,'model_checkpoint_best.pth.tar' ))
    if ((epoch+1)%10 == 0):
        plt.plot(np.array(epochs),np.array(losst),c='b',label='training')
        plt.plot(np.array(epochs),np.array(lossv),c='r',label='testing')
        plt.legend()
        if ((epoch+1) == nepoch):
            plt.savefig('%s/TrainvsValLoss_ep%d.png'%(plot_dir, epoch), bbox_inches='tight')
        plt.show()
